# Skipgram Model using MXNet/Gluon

## Introduction

This tutorials aims to teach how to implement word2vec skipgram model(negative sampling) with gluon interface.  
However note that python is not suitable for training word2vec model, data iterator in mxnet is limited to be single threaded and   additional trick is required for asynchronous SGD. This tutorial is for demonstration purpose only.  

## Other resources

For high performance library refer to  

 - cython version [Genism](https://radimrehurek.com/gensim/models/word2vec.html), 
 - or Original C version [Google](https://code.google.com/archive/p/word2vec/)

## CBOW version

Peopel who are interested in cbow model of word2vec(nce loss), refer to  
[CBOW](https://github.com/apache/incubator-mxnet/blob/master/example/nce-loss/wordvec.py)



## You will acquire the following:

- explain what skipgram model is
- implement custom blocks with Gluon Inteface, 
- extract, save and load parameters

## Prerequisites

To complete this tutorial, you need:

- [MXNet/Gluon](http://mxnet.io/get_started/setup.html#overview)
- [Language](https://www.python.org/)
- [Embedding Benchmark](https://github.com/kudkudak/word-embeddings-benchmarks)
- Familiarity with linear algebra, simple multi layer perceptron

In [ ]:
import time
import numpy as np
import logging
import sys, random, time, math
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet.gluon import Block, nn
from mxnet import autograd
import _pickle as cPickle
import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile
from six.moves import urllib
from six.moves import xrange  # pylint:

## The Data

[text8](http://mattmahoney.net/dc/textdata.html) **wikipedia dump**

In [ ]:
url = 'http://mattmahoney.net/dc/'
def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  local_filename = os.path.join(gettempdir(), filename)
  if not os.path.exists(local_filename):
    local_filename, _ = urllib.request.urlretrieve(url + filename,
                                                   local_filename)
  statinfo = os.stat(local_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception('Failed to verify ' + local_filename +
                    '. Can you get to it with a browser?')
  return local_filename

filename = maybe_download('text8.zip', 31344016)

split into list of words

In [ ]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = str(f.read(f.namelist()[0]))
  return data
buf = read_data(filename)
vocabulary = buf.split()

## Prepare the Data

In [ ]:
def build_dataset(words, n_words):
    dictionary = {}
    reverse_dictionary = ["NA"]
    count = [0] 
    data = [] 
    for word in vocabulary:
        if len(word) == 0:
            continue
        if word not in dictionary:
            dictionary[word] = len(dictionary) + 1
            count.append(0)
            reverse_dictionary.append(word)
        wid = dictionary[word]
        data.append(wid)
        count[wid] += 1
    negative = [] 
    for i, v in enumerate(count):
        if i == 0 or v < 5:
            continue
        v = int(math.pow(v * 1.0, 0.75))
        negative += [i for _ in range(v)]
    return data, count, dictionary, reverse_dictionary, negative
vocabulary_size = 50000
data, count, dictionary, reverse_dictionary, negative = build_dataset(vocabulary, vocabulary_size)

### From the raw text, we will build the followings

#### Dictionary
dictionary['apple'] returns integer index
#### Reverse dictionary 
reverse_dictionary['21'] returns corresponding word
#### Count
count['apple'] returns the occurences of apple in the corpus
#### Negative
this table will be used for sampling words in the training phase
#### Data
sequence of word is converted to sequence of indices, using dictionary

### Build Data Iterator

In [ ]:
class DataBatch(object):
    def __init__(self, data, label):
        self.data = data
        self.label = label

class Word2VecDataIterator(mx.io.DataIter):
    def __init__(self, batch_size=512, num_neg_samples=5, window=5):
        super(Word2VecDataIterator, self).__init__()
        self.batch_size = batch_size
        self.negative_samples = num_neg_samples
        self.window = window
        self.data, self.negative, self.dictionary = (data, negative, dictionary)

    @property
    def provide_data(self):
        return [('contexts', (self.batch_size, 1))]

    @property
    def provide_label(self):
        return [('targets', (self.batch_size, self.negative + 1))]

    def sample_ne(self):
        return self.negative[random.randint(0, len(self.negative) - 1)]

    def __iter__(self):
        input_data = []
        update_targets = []
        for pos, word in enumerate(self.data):
            for index in range(-self.window, self.window + 1):
                if (index != 0 and pos + index >= 0 and pos + index < len(self.data)):
                    context = self.data[pos + index]
                    if word != context:
                        input_data.append([word])
                        targets = []
                        targets.append(context) # positive sample
                        while len(targets) < self.negative_samples + 1: # negative sample
                            w = self.sample_ne()
                            if w != word:
                                targets.append(w)
                        update_targets.append(targets)

            # Check if batch size is full
            if len(input_data) > self.batch_size:
                batch_inputs = [mx.nd.array(input_data[:self.batch_size])]
                batch_update_targets = [mx.nd.array(update_targets[:self.batch_size])]
                yield DataBatch(batch_inputs, batch_update_targets)
                input_data = input_data[self.batch_size:]
                update_targets = update_targets[self.batch_size:]


![training-data](http://mccormickml.com/assets/word2vec/training_data.png)
## For example
if the size of the vocabulary is 10,  
then the size of the input and output layer is 10.  
(size, meaning number of unit in a layer)  
output will be something like this (0, 0.1, 0.7, 0.1, 0.1, 0, 0, 0, 0, 0)  

### Why 1 Hot Encoding?
1 hot encoded input will select only the corresponding row from the input weight matrix.  
**note** input weight matrix connects input and the hidden layer, this matrix is the vector we are trying to learn. (= word vectors)   

### Why Negative Sampling? 
At each training phase, we will not be calculating softmax loss over entire vocabulary.  
we will designate specific units to perform update on.  
thus a training sample will consists of context word that actually appears and five others that doesn't.   
negative samples are selected from the negative table we have built previously.  

e.g.)
"the quick"  
  
center : quick  
context : the  
  
postive sample (quick,the)  
negative sample (quick, fox)  
negative sample (quick, jump)  
negative sample (quick, cat)  
negative sample (quick, runs)  
negative sample (quick, of)  

In [ ]:
dictionary_size = len(reverse_dictionary)
batch_size = 512
num_hiddens = 100
num_negative_samples = 5

ctx = mx.gpu()
data_iterator = Word2VecDataIterator(batch_size=batch_size,
                                     num_neg_samples=num_negative_samples,
                                     window=5)

for batch in data_iterator:
    batches.append(batch)
    if (counting % 500 == 0):
        print(counting)
    counting = counting + 1

    cPickle.dump(training_data, open('all_batches.p', 'wb'))

## Create the Model

In [ ]:
class Model(gluon.HybridBlock):
    def __init__(self, **kwargs):
        super(Model, self).__init__(**kwargs)
        with self.name_scope():
            self.center = nn.Embedding(input_dim=dictionary_size,
                                       output_dim=num_hiddens,
                                       weight_initializer=mx.initializer.Uniform(1.0 / num_hiddens))

            self.target = nn.Embedding(input_dim=dictionary_size,
                                       output_dim=num_hiddens,
                                       weight_initializer=mx.initializer.Zero())

    def hybrid_forward(self, F, center, targets, labels):
        input_vectors = self.center(center)
        update_targets = self.target(targets)
        predictions = F.broadcast_mul(input_vectors, update_targets)
        predictions = F.sum(data=predictions, axis=2)
        sigmoid = F.sigmoid(predictions)
        loss = F.sum(labels * F.log(sigmoid) + (1 - labels) * F.log(1 - sigmoid), axis=1)
        loss = loss * -1.0 / batch_size
        loss_layer = F.MakeLoss(loss)
        return loss_layer
model = Model()
model.initialize(ctx=ctx)
model.hybridize()

![skipgram-architecture](http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png)
## Architecture
given a word, cbow and skipgram model both tries to predict a word.  
for skipgram the input is the **center** word, and output is the probability of a **context** word.  
input layer takes a single word(1 hot encoded) and output layer outputs probability of every word appearing in the context. 

### Center embedding
from the 1hot encoded input, we will be selecting word vectors from the weight matrix
only a single row will be selected
  
### Target embedding
from the target (postive sample, negative sample1,  negative sample2,  negative sample3,  negative sample4,  negative sample5)
only the 6 row will be selected
  
### Calculation

The skipgram architecture tries to predict the context given a word. The problem of predicting context words is framed as a set of independent binary classification tasks. Then the goal is to independently predict the presence (or absence) of context words. For the word at position $t$ we consider all context words as positive examples and sample negatives at random from the dictionary. For a chosen context position $c$, using the binary logistic loss, we obtain the following negative log-likelihood:

$$ \log (1 + e^{-s(w_t, w_c)}) +  \sum_{n \in \mathcal{N}_{t,c}}^{}{\log (1 + e^{s(w_t, n)})}$$

where $w_t$ is a center word, $w_c$ is a context word, $\mathcal{N}_{t,c}$ is a set of negative examples sampled from the vocabulary. By denoting the logistic loss function $l : x \mapsto \log(1 + e^{-x})$, we can re-write the objective as:

$$ \sum_{t=1}^{T}{ \sum_{c \in C_t}^{}{ \big[ l(s(w_t, w_c))} + \sum_{n \in \mathcal{N}_{t,c}}^{}{l(-s(w_t, n))}   \big]} $$

where $s(w_t, w_c) = u_{w_t}^T v_{w_c}$


## Fit the Model

In [ ]:
trainer = gluon.Trainer(model.collect_params(), 'sgd', {'learning_rate': 4, 'clip_gradient': 5})

labels = nd.zeros((batch_size, num_negative_samples + 1), ctx=ctx)
labels[:, 0] = 1 # [1 0 0 0 0 0]
start_time = time.time()
num_epochs = 10

def get_loss(epoch_n, batch_n, loss):
    if(batch_n==0 and epoch_n==0):
        loss = loss.asnumpy().sum()
    else:
        loss = .99 * loss + .01 * loss.asnumpy().sum()
    if(i + 1) % 100 == 0:
        print("%sth epoch , %sth batch. avg of loss: %s" % (epoch_n, batch_n, loss))
    return loss

for e in range(num_epochs):
    moving_loss = 0.
    for i, batch in enumerate(batches):
        center_words = batch.data[0].as_in_context(ctx)
        target_words = batch.label[0].as_in_context(ctx)
        with autograd.record():
            loss = model(center_words, target_words, labels)
        loss.backward()
        # ignore_stale_grad ; only update calculated target weights
        trainer.step(1, ignore_stale_grad=True)
        moving_loss = get_loss(e, i, moving_loss)
        if i > 15000:
            break
    print("1 epoch took %s seconds" % (time.time() - start_time))

We will use large batch size 512, update weights by sgd.  
**note** original c version of w2vec uses batch size 1, asynchronous sgd.  

## Save Word Vectors

In [ ]:
# format index : vector
key = list(model.collect_params().keys())
all_vecs = model.collect_params()[key[0]].data().asnumpy()
cPickle.dump(all_vecs, open('all_vecs.p', 'wb'))

#  foramt word : vector
w2vec_dict = dictionary.copy()
for word in dictionary:
    idx = dictionary[word]
    vector = all_vecs[idx]
    w2vec_dict[word] = vector

cPickle.dump(w2vec_dict, open('w2vec_dict.p', 'wb'))

## Evaluation

### T-SNE Visualization

In [ ]:
from sklearn.manifold import TSNE

num_points = 450

tsne = TSNE(perplexity=50, n_components=2, init='pca', n_iter=10000)
two_d_embeddings = tsne.fit_transform(all_vecs[:num_points])
labels = index_to_word[:num_points]

In [ ]:
from matplotlib import pylab

%matplotlib inline

def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(20,20))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

plot(two_d_embeddings, labels)

### Word Embedding
(optional) word embediing bench mark  
[Embedding Benchmark](https://github.com/kudkudak/word-embeddings-benchmarks)

In [ ]:
import logging
from web.datasets.analogy import fetch_google_analogy
from web.embeddings import fetch_SG_GoogleNews
import numpy as np
import _pickle as pickle

In [ ]:
w = load_embedding("w2vec_dict.p", format="dict")
data = fetch_google_analogy()

### analogy question

In [ ]:
# demo task1
subset = [1, 200, 1000, 3000, 5500]
for id in subset:
    w1, w2, w3 = data.X[id][0], data.X[id][1], data.X[id][2]
    print("Question: {} is to {} as {} is to ?".format(w1, w2, w3))
    print("Answer: " + data.y[id])
    print("Predicted: " + " ".join(w.nearest_neighbors(w[w2] - w[w1] + w[w3], exclude=[w1, w2, w3])))

### 5 nearest word

In [ ]:
# demo task2
targets = ["south", "usa", "eight", "earth"]
print("top 5 similar words list")
for target in targets:
    print("'"+target+"': "+str(w.nearest_neighbors(w[target], k = 5,  exclude = [target])))

### cosine similarity between two words

In [ ]:
# demo task3
print("cosine similarity measure")
word_pairs = [("france", "spain"),("newton", "apple"),("coke", "hamburger")]
for pair in word_pairs :
    cosine_similarity = np.dot(w[pair[0]], w[pair[1]])/(np.linalg.norm(w[pair[0]])* np.linalg.norm(w[pair[1]]))
    print("similarity score {} , {} : {} ".format(pair[0], pair[1] ,cosine_similarity))